In [1]:
!pip install turfpy
!pip install mysql.connector
!pip install geopandas
!pip install tqdm

  Created wheel for turfpy: filename=turfpy-0.0.7-py3-none-any.whl size=39119 sha256=2438ebed3ed31e7540b51def46cd0413c9a7e8d3d8da7ded82036b20d2dde6fb
  Stored in directory: /root/.cache/pip/wheels/56/d0/d8/13858feab5280185516cd22df1adc0f75426968815b057dc5e
Successfully built turfpy
     |████████████████████████████████| 11.9 MB 3.0 MB/s 
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp37-cp37m-linux_x86_64.whl size=247965 sha256=c3c77d64e73c771e3455df157d764f1669d6a8e7b71b38873ac085fd231ddb6a
  Stored in directory: /root/.cache/pip/wheels/42/2f/c3/692fc7fc1f0d8c06b9175d94f0fc30f4f92348f5df5af1b8b7
Successfully built mysql.connector
     |████████████████████████████████| 1.0 MB 5.1 MB/s 
     |████████████████████████████████| 6.3 MB 43.7 MB/s 
     |████████████████████████████████| 15.4 MB 27.5 MB/s 


In [2]:
from turfpy import measurement
from geojson import Point, Feature
from turfpy.transformation import intersect
from turfpy.measurement import length
import json
import mysql.connector
import geopandas as gpd
from tqdm import tqdm

In [15]:
mydb = mysql.connector.connect(
  host='mysql3.cs.stonybrook.edu',
  user='MothBalls',
  password='changeit',
  database='MothBalls',
  connection_timeout=3600
)

mycursor = mydb.cursor()

In [16]:
box_and_whisker_sql = "INSERT INTO box_and_whisker (\
id, \
basis, \
state \
) VALUES (%s,%s,%s)"

box_plot_sql = "INSERT INTO box_plot (\
id, \
basis, \
state, \
whisker_position, \
minimum, \
first_quartile, \
median, \
third_quartile, \
maximum \
) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"


stateNumber = '49'
stateName = 'utah'

boxPlot = json.load(open(f'{stateName}_box_whisker_data.json'))
basis = {"black" : 2, "vap_black": 10, "cvap_black": 18, "hispanic" : 3, "vap_hispanic": 11, "cvap_hispanic": 19, "american_indian": 4, "vap_american_indian": 12, "cvap_american_indian": 20, "asian": 5, "vap_asian": 13, "cvap_asian": 21, "hawaiian": 6, "vap_hawaiian": 14, "cvap_hawaiian": 22, "other": 7, "vap_other": 15, "cvap_other": 23, "democrat": 24, "republican": 25}

In [17]:
for key, value in basis.items():
  val_box_and_whisker = ((stateNumber + str(value)), \
                        value, \
                        stateNumber \
                        )
  mycursor.execute(box_and_whisker_sql, val_box_and_whisker)

  for position, box in enumerate(boxPlot[key], start=1):
    val_box_plot = ((stateNumber + str(value) + '_' + str(position)), \
                        value, \
                        stateNumber, \
                        position, \
                        box[0], \
                        box[1], \
                        box[2], \
                        box[3], \
                        box[4] \
                        )
    mycursor.execute(box_plot_sql, val_box_plot)

mydb.commit()
mydb.close()